# Processamento de Dados de Sensores Atmosféricos - Rio de Janeiro

Este notebook realiza a integração e o pré-processamento dos dados de sensores atmosféricos com as informações das estações de monitoramento, seguindo os passos:

1. **Unificação** dos datasets `sensores.zip` e `estacoes.csv` pela coluna `codnum`.
2. **Pré-processamento** filtrando apenas colunas relevantes e criando colunas temporais.
3. **Geração de arquivos individuais** para cada estação, visando facilitar a análise exploratória.

Fontes de dados:
- [Dados horários do monitoramento da qualidade do ar - MONITORA](https://www.data.rio/datasets/PCRJ::dados-hor%C3%A1rios-do-monitoramento-da-qualidade-do-ar-monitorar/about)
- [Localização das estações de monitoramento](https://www.data.rio/maps/5b1bf5c3e5114564bbf9b7a372b85e17/about)

## Importação de bibliotecas
Carrega as bibliotecas necessárias para manipulação e processamento dos dados.

In [1]:
from unidecode import unidecode
import pandas as pd
import requests 
import zipfile
import io
import os

## Unificação dos datasets

Carrega os dados de sensores e de estações, unindo-os pela coluna `codnum` para adicionar informações da estação a cada registro de leitura.

In [2]:
url_sensor = "https://github.com/AILAB-CEFET-RJ/qualiar/raw/main/data/DataRio/sensores.zip"

response = requests.get(url_sensor)
response.raise_for_status()

zip_file = zipfile.ZipFile(io.BytesIO(response.content))

print("Arquivos no ZIP:")
print(zip_file.namelist())

with zip_file.open(zip_file.namelist()[0]) as file:
    df_sensor = pd.read_csv(file)  

df_sensor.head()

Arquivos no ZIP:
['sensores.csv']


,objectid,data,codnum,estação,chuva,pres,rs,temp,ur,dir_vento,...,co,no,nox,o3,pm10,pm2_5,lat,lon,x_utm_sirgas2000,y_utm_sirgas2000
0,2372436,1/1/2011 6:30:00 AM,3,CA,0.2,1009.33,1.57,21.81,84.50,250.67,...,0.17,NaN,NaN,17.89,10.0,NaN,-22.908344,-43.178152,686853.7075,7.465470e+06
1,2372437,1/1/2011 7:30:00 AM,3,CA,0.0,1009.00,1.40,21.79,84.73,121.17,...,0.21,NaN,NaN,12.44,11.0,NaN,-22.908344,-43.178152,686853.7075,7.465470e+06
2,2372438,1/1/2011 8:30:00 AM,3,CA,0.2,1008.17,1.37,21.44,85.70,85.67,...,0.17,NaN,NaN,18.97,38.0,NaN,-22.908344,-43.178152,686853.7075,7.465470e+06
3,2372439,1/1/2011 9:30:00 AM,3,CA,0.0,1008.00,1.30,21.17,88.57,85.33,...,0.22,NaN,NaN,11.08,8.0,NaN,-22.908344,-43.178152,686853.7075,7.465470e+06
4,2372440,1/1/2011 10:30:00 AM,3,CA,0.2,1008.00,1.32,21.25,89.28,73.00,...,0.22,NaN,NaN,11.28,10.0,NaN,-22.908344,-43.178152,686853.7075,7.465470e+06


In [3]:
url_estacoes = 'https://raw.githubusercontent.com/EIC-BCC/25_2-QualiAr/refs/heads/main/data/DataRio/Estacoes.csv'

df_estacoes = pd.read_csv(url_estacoes, sep=',')

In [4]:
df_sensor['nome_estacao'] = df_sensor['codnum'].map(df_estacoes.set_index('codnum')['nome'])

df_sensor.head()

,objectid,data,codnum,estação,chuva,pres,rs,temp,ur,dir_vento,...,no,nox,o3,pm10,pm2_5,lat,lon,x_utm_sirgas2000,y_utm_sirgas2000,nome_estacao
0,2372436,1/1/2011 6:30:00 AM,3,CA,0.2,1009.33,1.57,21.81,84.50,250.67,...,NaN,NaN,17.89,10.0,NaN,-22.908344,-43.178152,686853.7075,7.465470e+06,ESTAÇÃO CENTRO
1,2372437,1/1/2011 7:30:00 AM,3,CA,0.0,1009.00,1.40,21.79,84.73,121.17,...,NaN,NaN,12.44,11.0,NaN,-22.908344,-43.178152,686853.7075,7.465470e+06,ESTAÇÃO CENTRO
2,2372438,1/1/2011 8:30:00 AM,3,CA,0.2,1008.17,1.37,21.44,85.70,85.67,...,NaN,NaN,18.97,38.0,NaN,-22.908344,-43.178152,686853.7075,7.465470e+06,ESTAÇÃO CENTRO
3,2372439,1/1/2011 9:30:00 AM,3,CA,0.0,1008.00,1.30,21.17,88.57,85.33,...,NaN,NaN,11.08,8.0,NaN,-22.908344,-43.178152,686853.7075,7.465470e+06,ESTAÇÃO CENTRO
4,2372440,1/1/2011 10:30:00 AM,3,CA,0.2,1008.00,1.32,21.25,89.28,73.00,...,NaN,NaN,11.28,10.0,NaN,-22.908344,-43.178152,686853.7075,7.465470e+06,ESTAÇÃO CENTRO


## Pré-processamento

Seleciona colunas relevantes, converte a coluna `data` para formato datetime, e cria colunas temporais para facilitar análises.

In [5]:
colunas_relevantes = ['nome_estacao', 'codnum', 'data', 'chuva', 'temp', 'ur', 'pres', 'rs', 'dir_vento', 'vel_vento', 'co', 'no', 'no2', 'nox', 'so2', 'o3', 'pm10', 'pm2_5', 'lat', 'lon']

df_sensor = df_sensor[colunas_relevantes]

In [6]:
df_sensor['data'] = pd.to_datetime(df_sensor['data'], format='%m/%d/%Y %I:%M:%S %p')
df_sensor['data_formatada'] = pd.to_datetime(df_sensor['data']).dt.strftime('%Y-%m-%d')
df_sensor['ano'] = df_sensor['data'].dt.year
df_sensor['mes'] = df_sensor['data'].dt.month

In [7]:
df_sensor = df_sensor[(df_sensor['ano'] > 2011) & (df_sensor['ano'] < 2025)]

df_sensor.head()

,nome_estacao,codnum,data,chuva,temp,ur,pres,rs,dir_vento,vel_vento,...,nox,so2,o3,pm10,pm2_5,lat,lon,data_formatada,ano,mes
8647,ESTAÇÃO CENTRO,3,2012-01-01 00:30:00,0.2,NaN,NaN,1008.25,0.73,123.83,1.25,...,NaN,NaN,NaN,NaN,NaN,-22.908344,-43.178152,2012-01-01,2012,1
8648,ESTAÇÃO CENTRO,3,2012-01-01 01:30:00,2.0,NaN,NaN,1008.70,0.00,81.50,1.55,...,NaN,NaN,NaN,NaN,NaN,-22.908344,-43.178152,2012-01-01,2012,1
8649,ESTAÇÃO CENTRO,3,2012-01-01 02:30:00,2.4,NaN,NaN,1008.75,0.00,89.33,1.38,...,NaN,NaN,NaN,NaN,NaN,-22.908344,-43.178152,2012-01-01,2012,1
17404,ESTAÇÃO COPACABANA,1,2012-01-01 00:30:00,0.0,25.77,81.57,1007.92,8.65,201.17,0.72,...,NaN,2.18,16.39,52.0,NaN,-22.965004,-43.180482,2012-01-01,2012,1
17405,ESTAÇÃO COPACABANA,1,2012-01-01 01:30:00,0.4,25.18,85.45,1008.53,0.95,278.00,0.70,...,NaN,1.73,19.38,51.0,NaN,-22.965004,-43.180482,2012-01-01,2012,1


## Geração de arquivos por estação

Cria uma pasta `Estacoes` e salva um arquivo CSV para cada estação, que será usado posteriormente na análise exploratória.

In [8]:
# Criar pasta para salvar os arquivos
os.makedirs('Estacoes', exist_ok=True)

# Gerar um CSV para cada estação
for estacao in df_sensor['nome_estacao'].unique():
    df_estacao = df_sensor[df_sensor['nome_estacao'] == estacao]
    
    nome_limpo = unidecode(estacao).replace(' ', '_')
    
    nome_arquivo = f"Estacoes/{nome_limpo}.csv"
    df_estacao.to_csv(nome_arquivo, index=False, encoding='utf-8')
    
    print(f"Arquivo salvo: {nome_arquivo} ({len(df_estacao)} registros)")

Arquivo salvo: Estacoes/ESTACAO_CENTRO.csv (113976 registros)
Arquivo salvo: Estacoes/ESTACAO_COPACABANA.csv (113976 registros)
Arquivo salvo: Estacoes/ESTACAO_SAO_CRISTOVAO.csv (113976 registros)
Arquivo salvo: Estacoes/ESTACAO_TIJUCA.csv (113976 registros)
Arquivo salvo: Estacoes/ESTACAO_BANGU.csv (113973 registros)
Arquivo salvo: Estacoes/ESTACAO_CAMPO_GRANDE.csv (113973 registros)
Arquivo salvo: Estacoes/ESTACAO_IRAJA.csv (113973 registros)
Arquivo salvo: Estacoes/ESTACAO_PEDRA_DE_GUARATIBA.csv (113889 registros)
